In [2]:
import os
from osgeo import gdal
import numpy as np
import subprocess
import sys

In [2]:
# average
main_folder = r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220706'
output_folder = r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220706\after_clip'
merged_file = r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220706\merged_Tmrt_2022_187_average.tif'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

specific_filename = 'Tmrt_average.tif'

In [3]:
def find_and_rename_tif_files(main_folder, specific_filename, output_folder):
    tif_files = []
    for root, dirs, files in os.walk(main_folder):
        for file in files:
            if file == specific_filename:
                folder_name = os.path.basename(root)
                new_filename = f'{folder_name}_{file}'
                output_file = os.path.join(output_folder, new_filename)
                tif_files.append((os.path.join(root, file), output_file))
    return tif_files

def crop_image(input_file, output_file, buffer=100):
    ds = gdal.Open(input_file)
    band = ds.GetRasterBand(1)
    data = band.ReadAsArray()
    
    cropped_data = data[buffer:-buffer, buffer:-buffer]
    
    gt = ds.GetGeoTransform()
    new_gt = (gt[0] + buffer * gt[1], gt[1], gt[2], gt[3] + buffer * gt[5], gt[4], gt[5])
    proj = ds.GetProjection()
    
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_file, 1000, 1000, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(new_gt)
    out_ds.SetProjection(proj)
    out_ds.GetRasterBand(1).WriteArray(cropped_data)
    out_ds.FlushCache()
    out_ds = None

tif_files = find_and_rename_tif_files(main_folder, specific_filename, output_folder)

cropped_files = []
for input_file, output_file in tif_files:
    crop_image(input_file, output_file)
    cropped_files.append(output_file)

vrt_options = gdal.BuildVRTOptions(separate=False)
vrt = gdal.BuildVRT('/vsimem/merged.vrt', cropped_files, options=vrt_options)
gdal.Translate(merged_file, vrt)

print(f'Merged image saved to {merged_file}')

Merged image saved to E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220716\merged_Tmrt_2022_197_average.tif


In [12]:
# 24h
main_folder = r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708'
output_folder = r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708\after_clip'
merged_output_folder = r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708\20220708_merged'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if not os.path.exists(merged_output_folder):
    os.makedirs(merged_output_folder)

specific_filenames = [
     f'Tmrt_2022_189_{i:04d}{"D" if 800 <= i <= 2000 else "N"}.tif' 
     for i in range(0, 2400, 100)
 ]

specific_filenames

['Tmrt_2022_189_0000N.tif',
 'Tmrt_2022_189_0100N.tif',
 'Tmrt_2022_189_0200N.tif',
 'Tmrt_2022_189_0300N.tif',
 'Tmrt_2022_189_0400N.tif',
 'Tmrt_2022_189_0500N.tif',
 'Tmrt_2022_189_0600N.tif',
 'Tmrt_2022_189_0700N.tif',
 'Tmrt_2022_189_0800D.tif',
 'Tmrt_2022_189_0900D.tif',
 'Tmrt_2022_189_1000D.tif',
 'Tmrt_2022_189_1100D.tif',
 'Tmrt_2022_189_1200D.tif',
 'Tmrt_2022_189_1300D.tif',
 'Tmrt_2022_189_1400D.tif',
 'Tmrt_2022_189_1500D.tif',
 'Tmrt_2022_189_1600D.tif',
 'Tmrt_2022_189_1700D.tif',
 'Tmrt_2022_189_1800D.tif',
 'Tmrt_2022_189_1900D.tif',
 'Tmrt_2022_189_2000D.tif',
 'Tmrt_2022_189_2100N.tif',
 'Tmrt_2022_189_2200N.tif',
 'Tmrt_2022_189_2300N.tif']

In [14]:
def find_and_rename_tif_files(main_folder, specific_filenames, output_folder):
    tif_files = {name: [] for name in specific_filenames}
    for root, dirs, files in os.walk(main_folder):
        for file in files:
            if file in specific_filenames:
                folder_name = os.path.basename(root)
                new_filename = f'{folder_name}_{file}'
                output_file = os.path.join(output_folder, new_filename)
                tif_files[file].append((os.path.join(root, file), output_file))
    return tif_files

def crop_image(input_file, output_file, buffer=100):
    try:
        ds = gdal.Open(input_file)
        if ds is None:
            print(f"Error: Unable to open input file {input_file}")
            return False
        band = ds.GetRasterBand(1)
        data = band.ReadAsArray()
        
        cropped_data = data[buffer:-buffer, buffer:-buffer]
        
        gt = ds.GetGeoTransform()
        new_gt = (gt[0] + buffer * gt[1], gt[1], gt[2], gt[3] + buffer * gt[5], gt[4], gt[5])
        proj = ds.GetProjection()
        
        driver = gdal.GetDriverByName('GTiff')
        out_ds = driver.Create(output_file, 1000, 1000, 1, gdal.GDT_Float32)
        if out_ds is None:
            print(f"Error: Unable to create output file {output_file}")
            return False
        out_ds.SetGeoTransform(new_gt)
        out_ds.SetProjection(proj)
        out_ds.GetRasterBand(1).WriteArray(cropped_data)
        out_ds.FlushCache()
        out_ds = None
        return True
    except Exception as e:
        print(f"Exception occurred while processing {input_file}: {e}")
        return False

tif_files_dict = find_and_rename_tif_files(main_folder, specific_filenames, output_folder)

for specific_filename, tif_files in tif_files_dict.items():
    cropped_files = []
    for input_file, output_file in tif_files:
        if crop_image(input_file, output_file):
            cropped_files.append(output_file)
    
    if len(cropped_files) == 0:
        print(f"No valid cropped files found for {specific_filename}")
        continue
    
    merged_file = os.path.join(merged_output_folder, f'merged_{specific_filename}')
    
    vrt_options = gdal.BuildVRTOptions(separate=False)
    vrt = gdal.BuildVRT('/vsimem/merged.vrt', cropped_files, options=vrt_options)
    if vrt is None:
        print(f"Error: Unable to build VRT for {specific_filename}")
        continue
    gdal.Translate(merged_file, vrt)
    
    print(f'Merged image saved to {merged_file}')

Merged image saved to E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708\20220708_merged\merged_Tmrt_2022_189_0000N.tif
Merged image saved to E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708\20220708_merged\merged_Tmrt_2022_189_0100N.tif
Merged image saved to E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708\20220708_merged\merged_Tmrt_2022_189_0200N.tif
Merged image saved to E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708\20220708_merged\merged_Tmrt_2022_189_0300N.tif
Merged image saved to E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708\20220708_merged\merged_Tmrt_2022_189_0400N.tif
Merged image saved to E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708\20220708_merged\merged_Tmrt_2022_189_0500N.tif
Merged image saved to E:\TAMU study\Projects\1